In [2]:
import numpy as np
import pandas as pd
import datetime

In [3]:
db_path = 'base_poluentes_internacoes_v6.CSV'
df = pd.read_csv(db_path, sep=';')

In [5]:
# filtrando internações ALTA OU MEDIA
inter_filter = df[df['internacao'] != 'baixa'].dropna()
inter_filter

,Data,"MP2,5",MP10,CO,SO2,NO2,O3,internacao
1,02/01/17,superior_media,superior_media,superior_media,inferior_media,superior_media,superior_media,alta
2,03/01/17,inferior_media,superior_media,superior_media,superior_media,superior_media,superior_media,media
3,04/01/17,inferior_media,superior_media,superior_media,inferior_media,inferior_media,superior_media,media
8,09/01/17,inferior_media,inferior_media,superior_media,superior_media,superior_media,inferior_media,media
9,10/01/17,superior_media,inferior_media,superior_media,superior_media,inferior_media,inferior_media,alta
10,11/01/17,inferior_media,inferior_media,superior_media,inferior_media,inferior_media,inferior_media,alta
16,17/01/17,superior_media,inferior_media,superior_media,inferior_media,superior_media,superior_media,media
17,18/01/17,superior_media,inferior_media,superior_media,inferior_media,superior_media,superior_media,media
18,19/01/17,superior_media,superior_media,inferior_media,inferior_media,inferior_media,inferior_media,media
19,20/01/17,inferior_media,superior_media,inferior_media,inferior_media,superior_media,superior_media,alta


In [26]:
# resumo
inter_filter.describe()

,Data,"MP2,5",MP10,CO,SO2,NO2,O3,internacao
count,37,37,37,37,37,37,37,37
unique,37,2,2,2,2,2,2,2
top,02/01/17,inferior_media,superior_media,superior_media,inferior_media,superior_media,superior_media,media
freq,1,20,26,34,34,32,19,20


In [27]:
db = {}  # irá armazenar os padrões em dias consecutivos
code = 0 # chave de acesso ao db
for index, row_i in inter_filter.iterrows():
    if index == 1:
        row_0 = row_i
        
    
    # salve os tempos iniciais e finais (b e f) do padrão
    b = datetime.datetime.strptime(row_0['Data'], '%d/%m/%y')
    f = datetime.datetime.strptime(row_i['Data'], '%d/%m/%y')
    diff = (f - b).days
    
    # padrão que não se repete em dias consecutivos
    pattern = [
        datetime.datetime.strftime(f, '%d/%m/%y'),
        datetime.datetime.strftime(f, '%d/%m/%y'),
        *row_i[1:]
    ]    
    
    # padrão que se repete em dias consecutivos
    if (row_i[1:].equals(row_0[1:])) and diff <= 1:
                    
        pattern = [
            datetime.datetime.strftime(b, '%d/%m/%y'),
            datetime.datetime.strftime(f, '%d/%m/%y'),
            *row_i[1:]
        ]
    else:
        code += 1
        
    # guarde o padrão no novo db
    if db.get(code, 0):
        db[code][1] = pattern[1]
    else:
        db[code] = pattern
    
    # passo
    row_0 = row_i
    
# transpor db para voltar ao formato convencional    
db = pd.DataFrame(db).transpose()    
new_col = list((inter_filter.columns).insert(1, 'f'))
new_col[0] = 'b'

# montar um DataFrame com os suportes de cada padrão encontrado em db
db = db.rename(columns={old:new for old, new in zip(db.columns, new_col)})
db_uniques = db.drop(columns=['b', 'f'])

supports = db.groupby(db_uniques.columns.tolist(),as_index=False).size()
supports['frequencia'] = (supports['size'] / db.shape[0]).round(3)

In [28]:
db

,b,f,"MP2,5",MP10,CO,SO2,NO2,O3,internacao
0,02/01/17,02/01/17,superior_media,superior_media,superior_media,inferior_media,superior_media,superior_media,alta
1,03/01/17,03/01/17,inferior_media,superior_media,superior_media,superior_media,superior_media,superior_media,media
2,04/01/17,04/01/17,inferior_media,superior_media,superior_media,inferior_media,inferior_media,superior_media,media
3,09/01/17,09/01/17,inferior_media,inferior_media,superior_media,superior_media,superior_media,inferior_media,media
4,10/01/17,10/01/17,superior_media,inferior_media,superior_media,superior_media,inferior_media,inferior_media,alta
5,11/01/17,11/01/17,inferior_media,inferior_media,superior_media,inferior_media,inferior_media,inferior_media,alta
6,17/01/17,18/01/17,superior_media,inferior_media,superior_media,inferior_media,superior_media,superior_media,media
7,19/01/17,19/01/17,superior_media,superior_media,inferior_media,inferior_media,inferior_media,inferior_media,media
8,20/01/17,20/01/17,inferior_media,superior_media,inferior_media,inferior_media,superior_media,superior_media,alta
9,22/01/17,22/01/17,inferior_media,superior_media,inferior_media,inferior_media,superior_media,superior_media,alta


In [29]:
supports

,"MP2,5",MP10,CO,SO2,NO2,O3,internacao,size,frequencia
0,inferior_media,inferior_media,superior_media,inferior_media,inferior_media,inferior_media,alta,1,0.048
1,inferior_media,inferior_media,superior_media,superior_media,superior_media,inferior_media,media,1,0.048
2,inferior_media,superior_media,inferior_media,inferior_media,superior_media,superior_media,alta,2,0.095
3,inferior_media,superior_media,superior_media,inferior_media,inferior_media,inferior_media,media,1,0.048
4,inferior_media,superior_media,superior_media,inferior_media,inferior_media,superior_media,media,1,0.048
5,inferior_media,superior_media,superior_media,inferior_media,superior_media,inferior_media,media,5,0.238
6,inferior_media,superior_media,superior_media,superior_media,superior_media,superior_media,media,1,0.048
7,superior_media,inferior_media,superior_media,inferior_media,superior_media,superior_media,alta,1,0.048
8,superior_media,inferior_media,superior_media,inferior_media,superior_media,superior_media,media,1,0.048
9,superior_media,inferior_media,superior_media,superior_media,inferior_media,inferior_media,alta,1,0.048
